## Importing the necessary packages

In [1]:
import numpy as np
from activation_function import sigmoid, relu, sigmoid_backward, relu_backward
import matplotlib.pyplot as plt
from matplotlib import style
import os, random, cv2, pickle

style.use('ggplot')

## Preprocessing and Storing the data in a Pickle file

In [ ]:
sets = ['train', 'test']
DATADIR = "D:\Dataset\intel-image-classification"
CATEGORIES = ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
training_data = []
testing_data = []
IMG_SIZE = 150

for s in sets:
    for category in CATEGORIES:
        path = os.path.join(DATADIR, str("seg_" + s), str("seg_" + s), category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try:
                img = os.path.join(path, img)
                img_array = cv2.imread(img)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                if s == "train":
                    training_data.append([new_array, class_num])
                elif s == "test":
                    testing_data.append([new_array, class_num])
#                 plt.imshow(new_array)  # To visualise the image
            except Exception as e:
                print(e)

random.shuffle(training_data)
random.shuffle(testing_data)
print(len(training_data), len(testing_data))

In [ ]:
X_train_list = []
y_train_list = []
X_test_list = []
y_test_list = []

for feature, label in training_data:
    X_train_list.append(feature)
    y_train_list.append(label)

for feature, label in testing_data:
    X_test_list.append(feature)
    y_test_list.append(label)

X_train_orig = np.array(X_train_list)
y_train_orig = np.array(y_train_list, ndmin = 2)
X_test_orig = np.array(X_test_list)
y_test_orig = np.array(y_test_list, ndmin = 2)

print(X_train_orig.shape, y_train_orig.shape, X_test_orig.shape, y_test_orig.shape)

In [ ]:
pickle_out_X = open('X_train.pickle', 'wb')
pickle.dump(X_train_orig, pickle_out_X)
pickle_out_X.close()

pickle_out_y = open('y_train.pickle', 'wb')
pickle.dump(y_train_orig, pickle_out_y)
pickle_out_y.close()

pickle_out_X = open('X_test.pickle', 'wb')
pickle.dump(X_test_orig, pickle_out_X)
pickle_out_X.close()

pickle_out_y = open('y_test.pickle', 'wb')
pickle.dump(y_test_orig, pickle_out_y)
pickle_out_y.close()

## Accessing the Data

In [2]:
pickle_in = open('X_train.pickle', 'rb')
X_train_orig = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('y_train.pickle', 'rb')
y_train_orig = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('X_test.pickle', 'rb')
X_test_orig = pickle.load(pickle_in)
pickle_in.close()

pickle_in = open('y_test.pickle', 'rb')
y_test_orig = pickle.load(pickle_in)
pickle_in.close()

In [3]:
# Exploring the dataset

print('Number of training examples:', X_train_orig.shape[0])
print('Number of testing examples:', X_test_orig.shape[0])
print('Each image is of size: (' + str(X_train_orig.shape[1]) + ", " + str(X_train_orig.shape[2]) + ", " + str(X_train_orig.shape[3]) + ")")
print('X_train_orig shape:', X_train_orig.shape)
print('y_train_orig shape:', y_train_orig.shape)
print('X_test_orig shape:', X_test_orig.shape)
print('y_test_orig shape:', y_test_orig.shape)

Number of training examples: 14034
Number of testing examples: 3000
Each image is of size: (150, 150, 3)
X_train_orig shape: (14034, 150, 150, 3)
y_train_orig shape: (1, 14034)
X_test_orig shape: (3000, 150, 150, 3)
y_test_orig shape: (1, 3000)


In [5]:
# Reshaping the training and testing examples

X_train_flatten = X_train_orig.reshape(X_train_orig.shape[0], -1)
X_test_flatten = X_test_orig.reshape(X_test_orig.shape[0], -1)

# Normalizing the data

X_train = X_train_flatten / 255.0
X_test = X_test_flatten / 255.0

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

X_train shape: (14034, 67500)
X_test shape: (3000, 67500)


In [ ]:
## Initializing the weights and biases for a 2-layer neural network

def initialize_parameters(n_x, n_h, n_y):
    """
        n_x - Size of input layer
        n_h - Size of hidden layer
        n_y - Size of output layer
    """
    W1 = np.random.randn(n_h, n_x) * 0.01
    b1 = np.zeros((n_h, 1))
    W2 = np.random_randn(n_y, n_h) * 0.01
    b2 = np.zeros((n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [6]:
## Initializing the weights and biases for an L-layer neural network

def initialize_parameters_deep(layer_dims):
    
    parameters = {}
    L = len(layer_dims)
    
    for l in range(1, L):
        parameters["W" + str(l)] = np.random.randn(layer_dims[l], layer_dims[l - 1])
        parameters["b" + str(l)] = np.zeros((layer_dims[l], 1))
    
    return parameters

In [7]:
## Implementing the basic forward propagation algorithm: Z = W.T + b for a 2-layer Neural Network

def linear_forward(W, A, b):
    
    Z = np.dot(W, A) + b
    cache = (A, W, b)
    
    return Z, cache

In [8]:
## Implementing the necessary activation function to Z (received from linear_forward function)

def linear_activation_forward(A_prev, W, b, activation):
    
    if activation == "sigmoid":
        Z, linear_cache = linear_forward(W, A_prev, b)
        A, activation_cache = sigmoid(Z)
    
    elif activation == "relu":
        Z, linear_cache = linear_forward(W, A_prev, b)
        A, activation_cache = relu(Z)
    
    return A, cache

In [9]:
## Implementing forward propagation for an L-layer Neural Network

def L_model_forward(X, parameters):
    
    L = len(parameters) / 2
    A = X
    caches = []
    
    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters["W" + str(l)], parameters["b" + str(l)], activation = "relu")
        caches.append(cache)
    
    AL, cache = linear_activation_forward(A, parameters["W" + str(L)], parameters["W" + str(L)], activation = "sigmoid")
    caches.append(cache)
    
    return AL, caches

In [10]:
## Calculating the cost function

def compute_cost(AL, y):
    
    m = y.shape[1]        # number of examples
    cost = - (1 / m) * np.sum(np.multiply(np.log(AL), y) + np.multiply(np.log(1 - AL), (1 - y)))
    
    cost = np.squeeze(cost)  # Checking the shape of cost; Converts [[21]] to 21
    
    return cost

In [11]:
## Implementing the backward propagation algorithm:

def linear_backward(dZ, cache):
    
    A_prev, W, b = cache
    m = A_prev.shape[1]
    
    dW = (1 / m) * np.dot(dZ, A_prev.T)
    db = (1 / m) * np.sum(dZ, axis = 1, keepdims = True)     # keepdims = True changes a vector of dimension (n,) to (n, 1)
    dA_prev = np.dot(W.T, dZ)
    
    return dA_prev, dW, db

In [ ]:
def linear_activation_backward(dA, cache, activation):
    
    linear_cache, activation_cache = cache
    
    if activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    elif activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
        
    return dA_prev, dW, db